<a href="https://colab.research.google.com/github/tamirdh/Critics_website_project/blob/master/image_captioning_deep_learning_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloads

In [ ]:
%cd /content
!rm -rf coco
!rm -rf sample_data
!mkdir coco
%cd coco
!mkdir images
%cd images

!wget -c http://images.cocodataset.org/zips/train2017.zip
#!wget -c http://images.cocodataset.org/zips/val2017.zip

!unzip train2017.zip
#!unzip val2017.zip

!rm train2017.zip
#!rm val2017.zip

%cd ../
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip

!unzip annotations_trainval2017.zip

!rm annotations_trainval2017.zip


/content
/content/coco
/content/coco/images
--2021-06-01 10:47:44--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.36.124
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.36.124|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip        18%[==>                 ]   3.41G  15.0MB/s    eta 16m 19s

# Data Loading

## Imports and Vocabulary 

In [ ]:
import os
from collections import Counter
import spacy
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from pycocotools.coco import COCO
from PIL import Image
import pickle

spacy_eng = spacy.load("en")

class Vocabulary:
    def __init__(self,freq_threshold):
        #setting the pre-reserved tokens int to string tokens
        # PAD- padding symbol
        # SOS- Start of Sentence
        # EOS- end of sentence
        # UNK- unknown word (unknown\ below threshold)
        self.itos = {0:"<PAD>",1:"<SOS>",2:"<EOS>",3:"<UNK>"}
        #string to int tokens
        #its reverse dict self.itos
        self.stoi = {v:k for k,v in self.itos.items()}
        self.freq_threshold = freq_threshold
        
    def __len__(self):
      return len(self.itos)
    
    @staticmethod
    def tokenize(text):
        return [token.text.lower() for token in spacy_eng.tokenizer(text)]
    
    def build_vocab(self, sentence_list):
        frequencies = Counter()
        idx = 4
        for sentence in sentence_list:

            for word in self.tokenize(sentence):
                frequencies[word] += 1
                
                #add the word to the vocab if it reaches minum frequecy threshold
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    if idx > 0 and idx % 1000==0:
                        print(f"Added {idx} words to vocab")
                    idx += 1
            

        print(f"Done, added {idx-1} words to vocabulary")
    
    def numericalize(self,text):
        """ For each word in the text corresponding index token for that word form the vocab built as list """
        tokenized_text = self.tokenize(text)
        result = [ self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text ]
        return result

## Dataset custom class

In [ ]:
import pickle
class COCODataset(Dataset):
    """
    COCODataset
    """
    def __init__(self,root_dir,annotation_file,transform=None,freq_threshold=5,
                 load_vocab=False, vocab_loc = "vocab.pkl"):
      """
      can use load_vocab to use a previously created vocabulary (time saving feature)
      freq_threshold: words with a count below this number will be marked as <UNK>
      """
      self.root_dir = root_dir
      self.coco = COCO(annotation_file)
      self.transform = transform
      self.cap_max_size = 0
      #Get image and caption colum from the dataframe
      self.imgs = []
      self.captions = []
      for idx,ann in enumerate(self.coco.anns.values()):
        self.imgs.append(self.coco.loadImgs((ann['image_id']))[0]["file_name"])
        self.captions.append(ann['caption'])
        if (idx) % 1000 == 0 and idx>0:
          print(f"Processed {idx} images and captions")
      print("Finished processing images and captions")
      print(f"Got:{len(set(self.imgs))} pictures with {len(self.captions)} captions!")
      
      #Initialize vocabulary and build vocab
      if load_vocab:
        with open(vocab_loc, "rb") as source:
          self.vocab = pickle.load(source)
        print(f"Loaded vocabulary from {vocab_loc}")
      
      else:
        print("Build vocabulary")
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocab(self.captions)
        print("Finished building vocabulary")
        with open(vocab_loc, "wb") as dest:
          pickle.dump(self.vocab, dest)
      
      print(f"Using {len(self.vocab)} words")
    
    def __len__(self):
      return len(self.imgs)
    
    def __getitem__(self,idx):
      caption = self.captions[idx]
      img_name = self.imgs[idx]
      img_location = os.path.join(self.root_dir,img_name)
      img = Image.open(img_location).convert("RGB")
      
      #apply the transfromation to the image
      if self.transform:
          img = self.transform(img)
      
      #numericalize the caption text
      caption_vec = [self.vocab.stoi["<SOS>"]]
      caption_vec.extend(self.vocab.numericalize(caption))
      caption_vec.append(self.vocab.stoi["<EOS>"])
      
      return img, torch.tensor(caption_vec,dtype=torch.long)

## Dataloader creation

In [ ]:
# define a transformation to add some noise and variance to our images
transformation = transforms.Compose([transforms.Resize((512,512), Image.NEAREST),
                                     transforms.ToTensor(),
                                     transforms.RandomInvert(),
                                     transforms.RandomVerticalFlip(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                      ])

In [ ]:
class CapsCollate:
    """
    Collate to apply the padding to the captions with dataloader
    """
    def __init__(self,pad_idx,batch_first=False, vec_len=-1):
        self.pad_idx = pad_idx
        self.batch_first = batch_first
        self.vec_len = vec_len + 2 # adding the <SOS> and <EOS>
        assert self.vec_len > 0, "Vector length must be positive integer"
    
    def __call__(self,batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs,dim=0)
        targets_list = list()
        for item in batch:
            # item = (img:Image, caption:tensor)
            addition = self.vec_len-len(item[1])
            padded_target = torch.cat((item[1], torch.empty(addition,dtype=torch.long).fill_(pad_idx)),dim=0)
            targets_list.append(padded_target)
            #print(f"GOT:{item[1]}, {item[1].type()}\nAdding:{addition}\nPADDED:{padded_target}\n{padded_target.type()}")
        targets = torch.stack(targets_list,0)
        #print(f"Targets shape:{targets.shape}")
        return imgs,targets

In [ ]:
#writing the dataloader
#setting the constants
BATCH_SIZE = 4
NUM_WORKER = 1

dataset =  COCODataset(
    root_dir = "/content/coco/images/train2017",
    annotation_file= "/content/coco/annotations/captions_train2017.json",
    transform=transformation,
    freq_threshold=5,
    load_vocab=False
)

In [ ]:

#token to represent the padding
pad_idx = dataset.vocab.stoi["<PAD>"]

data_loader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=True,
    collate_fn=CapsCollate(pad_idx=pad_idx,batch_first=True, vec_len=50)
)

In [ ]:
for i,j in data_loader:
  break

# Models

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

device = "cuda" if torch.cuda.is_available() else "cpu"

class EncoderCNN(nn.Module):
  def __init__(self, embed_size, train_CNN=False):
      super(EncoderCNN, self).__init__()
      self.train_CNN = train_CNN
      self.inception = models.inception_v3(pretrained=True, aux_logits=False)
      self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
      self.relu = nn.ReLU()
      self.dropout = nn.Dropout(0.5)

  def forward(self, images):
      
      features = self.inception(images)
      output = self.dropout(self.relu(features))
      return output


class DecoderRNN(nn.Module):
  """
  Input is a CNN network, output will be a caption.
  TODO: Check how to implement a transformer for better results
  """
  def __init__(self, embed_size, hidden_size, vocab_size):
      super(DecoderRNN, self).__init__()
      self.hidden_size = hidden_size
      self.vocab_size = vocab_size
      self.embed = nn.Embedding(vocab_size, embed_size)
      self.lstm_cell = nn.LSTMCell(embed_size, hidden_size)
      self.fc_out = nn.Linear(hidden_size, vocab_size)
      self.dropout = nn.Dropout(0.5)

  def forward(self, features, captions):
        # batch size
        batch_size = features.size(0)
        
        
        # init the hidden and cell states to zeros
        hidden_state = torch.zeros((batch_size, self.hidden_size)).to(device)
        cell_state = torch.zeros((batch_size, self.hidden_size)).to(device)
    
        # define the output tensor placeholder
        outputs = torch.empty((batch_size, captions.size(1), self.vocab_size)).to(device)

        # embed the captions
        captions_embed = self.embed(captions)
        # tensor of shape (B, LEN, EMBED SIZE)
        # LEN- vectors length (longest caption+2)
        
        # pass the caption word by word
        for t in range(captions.size(1)):

            # for the first time step the input is the feature vector
            if t == 0:
                hidden_state, cell_state = self.lstm_cell(features, (hidden_state, cell_state))
                
            # for the 2nd+ time step, using teacher forcer
            else:
                #hidden_state, cell_state = self.lstm_cell()
                hidden_state, cell_state = self.lstm_cell(captions_embed[:, t, :], (hidden_state, cell_state))
            
            # output of the attention mechanism
            out = self.fc_out(self.dropout(hidden_state))
            # build the output tensor
            outputs[:, t, :] = out
        return outputs

  

class CNNtoRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, train_CNN=False):
      super(CNNtoRNN, self).__init__()
      self.encoderCNN = EncoderCNN(embed_size, train_CNN).to(device)
      self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size).to(device)

  def forward(self, images, captions):
      features = self.encoderCNN(images)
      outputs = self.decoderRNN(features, captions)
      return outputs
  def caption_images(self, features, vocab, max_len=50):
      # Inference part
      # Given the image features generate the captions
      # input shape: (3,x,y) where, x,y: image size
      # ouput: captions list
      self.eval()
      with torch.no_grad():
        image_pred = self.encoderCNN(features)
        batch_size = features.size(0)     
        # init the hidden and cell states to zeros
        hidden_state = torch.zeros((batch_size, self.decoderRNN.hidden_size)).to(device)
        cell_state = torch.zeros((batch_size, self.decoderRNN.hidden_size)).to(device)   
        captions_embed = None # embedding of partial caption
        #starting input
        captions = []
        for t in range(max_len):
            
           # for the first time step the input is the feature vector
            if t == 0:
                hidden_state, cell_state = self.decoderRNN.lstm_cell(image_pred, (hidden_state, cell_state))    
            # for the 2nd+ time step, use previously generated caption
            else:
                hidden_state, cell_state = self.decoderRNN.lstm_cell(captions_embed, (hidden_state, cell_state))
            
            # output of the attention mechanism
            out = self.decoderRNN.fc_out(self.decoderRNN.dropout(hidden_state))
            word_idx = torch.argmax(out).item()
            captions.append(word_idx)
            if vocab.itos[word_idx] == vocab.stoi["<EOS>"]:
              break
            captions_embed = self.decoderRNN.embed(torch.argmax(out)).unsqueeze(0)
            
            
            # build the output tensor
        print(captions)
        #covert the vocab idx to words and return sentence
        self.train()
        return [vocab.itos[idx] for idx in captions if idx != vocab.stoi["<PAD>"]]

  

# Training

## training function

In [ ]:
import torch.optim as optim
from tqdm import tqdm


def train(max_epochs, model):
  # Hyperparameters
  learning_rate = 3e-4
  device = "cuda" if torch.cuda.is_available() else "cpu"

  
  # init model
  model = model.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  model.train()
     

  # start epochs
  for epoch in range(max_epochs):
    for idx, (img, captions) in tqdm(
            enumerate(data_loader), total=len(data_loader), leave=False
        ):
      img = img.to(device)
      captions = captions.to(device).long()
      output = model(img, captions).to(device)
      loss = criterion(output.reshape(-1, output.shape[2]), captions.reshape(-1))
      optimizer.zero_grad()
      loss.backward(loss)
      optimizer.step()

      if idx == 2:
        dataiter = iter(data_loader)
        img_show,cap = next(dataiter)
        print(f"Loss {loss.item():.5f}\n")
        demo_cap = model.caption_images(img_show[0:1].to(device), vocab=dataset.vocab, max_len=30)
        demo_cap = ' '.join(demo_cap)
        print("Predicted")
        show_image(img_show[0],title=demo_cap)
        print("Original")
        cap = cap[0]
        print(cap.long())
        demo_cap = ' '.join([dataset.vocab.itos[idx2.item()] for idx2 in cap if idx2.item() != dataset.vocab.stoi["<PAD>"]])
        show_image(img_show[0],title=demo_cap, transform=False)
        #input("Contiue?")
        break
      if idx > 20:
        break
    
        

        

  return model

## image function

In [ ]:
import matplotlib.pyplot as plt
def show_image(img, title=None, transform=True):
    """Imshow for Tensor."""
    
    #unnormalize 
    if transform:
      img[0] = img[0] * 0.229
      img[1] = img[1] * 0.224 
      img[2] = img[2] * 0.225 
      img[0] += 0.485 
      img[1] += 0.456 
      img[2] += 0.406
      
    img = img.numpy().transpose((1, 2, 0))
    
    
    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

## Process

In [ ]:
embed_size = 1024
hidden_size = 512
vocab_size = len(dataset.vocab)
model = CNNtoRNN(embed_size, hidden_size, vocab_size, train_CNN=False)
trained_model = train(3)

# Misc

In [ ]:
v = Vocabulary(freq_threshold=1)

v.build_vocab(["This is a good place to find a city"])
print(v.stoi)
print(v.numericalize("This is a good place to find a city here!!"))

In [ ]:
for img,caption in data_loader:
  print(img.size())
  captions = caption.unsqueeze(2)
  print(captions.shape)
  cap = caption[0]
  print(cap.size())
  print(cap)
  print([idx for idx in cap])
  print(dataset.vocab.itos[cap[2].item()])
  demo_cap = ' '.join([dataset.vocab.itos.get(idx.item(), "<UNK>") for idx in cap if idx.item()!=dataset.vocab.stoi["<PAD>"]])
  show_image(img[0],title=demo_cap)
  break

In [ ]:
len(dataset.vocab)

In [ ]:
a = torch.tensor([1,2])
a.shape



In [ ]:
a.unsqueeze(0).unsqueeze(0).shape


In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl

In [ ]:
dev = xm.xla_device()
dev